# 식 생성을 위한 코드

목차
1. [모듈 불러오기](#1-모듈-불러오기)
2. [설정 값 세팅](#2-설정-값-세팅)
3. [변수 생성](#3-변수-생성)
4. [식 생성](#4-식-생성)
5. [Degree Reduction](#5-degree-reduction)
6. [dimod 모듈의 적합한 형태로 변화 및 데이터 저장](#6-dimod-모듈의-적합한-형태로-변환-및-데이터-저장)

#### 1. 모듈 불러오기

In [41]:
import math
import sympy as sym
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle

#### 2. 설정 값 세팅

In [42]:
TABLE_BIT = 2 # 수업 종류를 나타내는 비트 수
TABLE_WIDTH = 2 # 시간표 가로
TABLE_LENGTH = 2 # 시간표 세로
TABLE_COUNT = 2 # 반의 수(시간표 개수)


CLASS_LIST = {  # 수업 시수
  'free':2,
  'class1':1,
  'none1':1,
  'none2':0.
  # (수업 이름):(수업 횟수)
}

CLASS_MILD_STEEL_LIST = {  # 연강
  'free':2,
  # (수업 이름):(수업 횟수)
}

#### 3. 변수 생성

In [43]:
var_list = []
for i in range(0, TABLE_BIT * TABLE_COUNT * TABLE_LENGTH * TABLE_WIDTH):
  var_list.append(sym.symbols('v' + str(i)))
class_bit_count = math.ceil(math.log(len(CLASS_LIST),2))
print(var_list)

[v0, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15]


#### 4. 식 생성

In [44]:
def find_variable(table_num, loc, bit):
  return var_list[table_num * TABLE_WIDTH * TABLE_LENGTH * TABLE_BIT + loc * TABLE_BIT + bit]

def num_to_binary_list(num):
  a = list(0 for i in range(TABLE_BIT))
  for i in range(TABLE_BIT):
    a[i] = num % 2
    num = num // 2
  return a

In [ ]:
poly = sym.sympify("0")

# 겹치는 부분 확인
for i in range(0, TABLE_COUNT):
  for j in range(i+1, TABLE_COUNT):
    for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
      new_poly = sym.sympify("1")
      for m in range(0, TABLE_BIT):
        new_poly *= ((1-find_variable(i, k, m)) * (1-find_variable(j, k, m))+find_variable(i, k, m) * find_variable(j, k, m))
      sum_poly = 0
      for m in range(0, TABLE_BIT):
        sum_poly += (find_variable(i, k, m) + find_variable(j, k, m))
      new_poly *= sum_poly
      poly += new_poly
poly = poly.expand()

# 모든 변수는 0또는 1의 값을 가지기 때문에 거듭제곱은 의미가 없음
# 그거 지우는 부분 코드임
new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
del new_poly
del sum_poly



# 수업 시수가 맞게 있는지 확인
for i in range(0, TABLE_COUNT):
  for j in range(1, 2**TABLE_BIT):
    new_poly = 0
    class_bit_list = num_to_binary_list(j)
    for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
      sum_poly = 1
      for bit in range(0, TABLE_BIT):
        sum_poly *= ((1-find_variable(i, k, bit))*(1-class_bit_list[bit])+find_variable(i, k, bit) * class_bit_list[bit])
      new_poly += sum_poly
    poly += (new_poly - list(CLASS_LIST.values())[j]) ** 2
    poly = poly.expand()
    poly = poly.simplify()
new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
del new_poly
del sum_poly



new_poly_a = sym.sympify("0")
for a in CLASS_MILD_STEEL_LIST:  # 각 수업에 대해
  new_poly_b = sym.sympify("0")
  check_class = CLASS_MILD_STEEL_LIST[a]
  binary_list = num_to_binary_list(list(CLASS_LIST.keys()).index(a))
  
  for b in range(0, TABLE_COUNT):  # 각 반 시간표에 대해
    new_poly_c = sym.sympify("0")
    
    for c in range(0, TABLE_WIDTH):  # 각 가로줄에 대해
      new_poly_d = sym.sympify("0")
      
      for d in range(0, TABLE_LENGTH - check_class + 1):  # 각 세로줄에 대해
        new_poly_e = sym.sympify("1")
        
        for bit in range(0, TABLE_BIT):  # 각 비트에 대해                            # 연강 확인 코드
          new_poly_e1 = sym.sympify("1")                                       # ----------------------------------------
          new_poly_e2 = sym.sympify("1")
          
          for e in range(0, check_class):  # 연강 확인 수
            new_poly_e1 *= find_variable(b, c * TABLE_LENGTH + d + e, bit)
          new_poly_e1 *= binary_list[bit]
          
          for e in range(0, check_class):
            new_poly_e2 *= (1 - find_variable(b, c * TABLE_LENGTH + d + e, bit))
          new_poly_e2 *= (1 - binary_list[bit])                                    # ----------------------------------------
          
          new_poly_e *= (new_poly_e1 + new_poly_e2)
        
        new_poly_d = (new_poly_d + new_poly_e) - (new_poly_d * new_poly_e)
      new_poly_c = (new_poly_c + new_poly_d) - (new_poly_c * new_poly_d)
    new_poly_b += (1 - new_poly_c)
  new_poly_a += new_poly_b
poly += new_poly_a

new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
poly = poly.expand()
del new_poly
del new_poly_a
del new_poly_b
del new_poly_c
del new_poly_d
del new_poly_e
del new_poly_e1
del new_poly_e2

print(poly)
f = open('poly.txt', 'w')
f.write(str(poly))
f.close()

In [45]:
# poly = sym.sympify("0")

# # 겹치는 부분 확인
# for i in range(0, TABLE_COUNT):
#   for j in range(i+1, TABLE_COUNT):
#     for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
#       new_poly = sym.sympify("1")
#       for m in range(0, TABLE_BIT):
#         new_poly *= ((1-find_variable(i, k, m)) * (1-find_variable(j, k, m))+find_variable(i, k, m) * find_variable(j, k, m))
#       sum_poly = 0
#       for m in range(0, TABLE_BIT):
#         sum_poly += (find_variable(i, k, m) + find_variable(j, k, m))
#       new_poly *= sum_poly
#       poly += new_poly
# poly = poly.expand()

# # 모든 변수는 0또는 1의 값을 가지기 때문에 거듭제곱은 의미가 없음
# # 그거 지우는 부분 코드임
# new_poly = str(poly)
# new_poly = re.sub('\*\*\d*', '', new_poly)
# poly = sym.sympify(new_poly)
# poly = poly.simplify()
# del new_poly
# del sum_poly



# # 수업 시수가 맞게 있는지 확인
# for i in range(0, TABLE_COUNT):
#   for j in range(0, 2**TABLE_BIT):
#     new_poly = 0
#     class_bit_list = num_to_binary_list(j)
#     for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
#       sum_poly = 1
#       for bit in range(0, TABLE_BIT):
#         sum_poly *= ((1-find_variable(i, k, bit))*(1-class_bit_list[bit])+find_variable(i, k, bit) * class_bit_list[bit])
#       new_poly += sum_poly
#     poly += (new_poly - list(CLASS_LIST.values())[j]) ** 2
#     poly = poly.expand()
#     poly = poly.simplify()
# new_poly = str(poly)
# new_poly = re.sub('\*\*\d*', '', new_poly)
# poly = sym.sympify(new_poly)
# poly = poly.simplify()
# del new_poly
# del sum_poly
# print(poly)

8*v0*v1*v2*v3 - 4*v0*v1*v2 - 4*v0*v1*v3 + 8*v0*v1*v4*v5 - 4*v0*v1*v4 - 4*v0*v1*v5 + 8*v0*v1*v6*v7 - 4*v0*v1*v6 - 4*v0*v1*v7 + 8*v0*v1*v8*v9 - 2*v0*v1*v8 - 2*v0*v1*v9 + 6*v0*v1 - 4*v0*v2*v3 + 4*v0*v2 + 2*v0*v3 - 4*v0*v4*v5 + 4*v0*v4 + 2*v0*v5 - 4*v0*v6*v7 + 4*v0*v6 + 2*v0*v7 - 2*v0*v8*v9 + 2*v0*v8 - 4*v0 - 4*v1*v2*v3 + 2*v1*v2 + 4*v1*v3 - 4*v1*v4*v5 + 2*v1*v4 + 4*v1*v5 - 4*v1*v6*v7 + 2*v1*v6 + 4*v1*v7 - 2*v1*v8*v9 + 2*v1*v9 - 4*v1 + 8*v10*v11*v12*v13 - 4*v10*v11*v12 - 4*v10*v11*v13 + 8*v10*v11*v14*v15 - 4*v10*v11*v14 - 4*v10*v11*v15 + 8*v10*v11*v2*v3 - 2*v10*v11*v2 - 2*v10*v11*v3 + 8*v10*v11*v8*v9 - 4*v10*v11*v8 - 4*v10*v11*v9 + 6*v10*v11 - 4*v10*v12*v13 + 4*v10*v12 + 2*v10*v13 - 4*v10*v14*v15 + 4*v10*v14 + 2*v10*v15 - 2*v10*v2*v3 + 2*v10*v2 - 4*v10*v8*v9 + 4*v10*v8 + 2*v10*v9 - 4*v10 - 4*v11*v12*v13 + 2*v11*v12 + 4*v11*v13 - 4*v11*v14*v15 + 2*v11*v14 + 4*v11*v15 - 2*v11*v2*v3 + 2*v11*v3 - 4*v11*v8*v9 + 2*v11*v8 + 4*v11*v9 - 4*v11 + 8*v12*v13*v14*v15 - 4*v12*v13*v14 - 4*v12*v13*v15 + 8*

In [46]:
# for c in CLASS_MILD_STEEL_LIST:
#   class_bit_list = num_to_binary_list(CLASS_NUM_LIST[c])
#   sum_poly5 = 0
#   for _ in range(0, TABLE_COUNT):
#     sum_poly4 = 0
#     for i in range(0, TABLE_WIDTH):
#       check_class = CLASS_MILD_STEEL_LIST[c]
#       line_sum_poly = 0
#       for j in range(0, TABLE_LENGTH - check_class + 1):
#         sum_poly3 = 1
#         for k in range(0, TABLE_BIT):
#           sum_poly1 = 1
#           for m in range(0, check_class):
#             sum_poly1 *= find_variable(_, TABLE_LENGTH * i + j + m, k)
#           sum_poly1 *= class_bit_list[k]
#           sum_poly2 = 1
#           for m in range(0, check_class):
#             sum_poly2 *= (1-find_variable(_, TABLE_LENGTH * i + j + m, k))
#           sum_poly2 *= (1-class_bit_list[k])
#           sum_poly3 *= (sum_poly1 + sum_poly2)
#         line_sum_poly += sum_poly3
#       sum_poly4 += (1 - line_sum_poly)
#     sum_poly5 += sum_poly4
#   poly += sum_poly5
  
# new_poly = str(poly)
# new_poly = re.sub('\*\*\d*', '', new_poly)
# poly = sym.sympify(new_poly)
# poly = poly.simplify()
# # poly *= sum_poly6
# poly = poly.expand()
# print(poly)

# f = open('poly.txt', 'w')
# f.write(str(poly))
# f.close()

7*v0*v1*v2*v3 - 3*v0*v1*v2 - 3*v0*v1*v3 + 8*v0*v1*v4*v5 - 4*v0*v1*v4 - 4*v0*v1*v5 + 8*v0*v1*v6*v7 - 4*v0*v1*v6 - 4*v0*v1*v7 + 8*v0*v1*v8*v9 - 2*v0*v1*v8 - 2*v0*v1*v9 + 5*v0*v1 - 3*v0*v2*v3 + 3*v0*v2 + v0*v3 - 4*v0*v4*v5 + 4*v0*v4 + 2*v0*v5 - 4*v0*v6*v7 + 4*v0*v6 + 2*v0*v7 - 2*v0*v8*v9 + 2*v0*v8 - 3*v0 - 3*v1*v2*v3 + v1*v2 + 3*v1*v3 - 4*v1*v4*v5 + 2*v1*v4 + 4*v1*v5 - 4*v1*v6*v7 + 2*v1*v6 + 4*v1*v7 - 2*v1*v8*v9 + 2*v1*v9 - 3*v1 + 8*v10*v11*v12*v13 - 4*v10*v11*v12 - 4*v10*v11*v13 + 8*v10*v11*v14*v15 - 4*v10*v11*v14 - 4*v10*v11*v15 + 8*v10*v11*v2*v3 - 2*v10*v11*v2 - 2*v10*v11*v3 + 7*v10*v11*v8*v9 - 3*v10*v11*v8 - 3*v10*v11*v9 + 5*v10*v11 - 4*v10*v12*v13 + 4*v10*v12 + 2*v10*v13 - 4*v10*v14*v15 + 4*v10*v14 + 2*v10*v15 - 2*v10*v2*v3 + 2*v10*v2 - 3*v10*v8*v9 + 3*v10*v8 + v10*v9 - 3*v10 - 4*v11*v12*v13 + 2*v11*v12 + 4*v11*v13 - 4*v11*v14*v15 + 2*v11*v14 + 4*v11*v15 - 2*v11*v2*v3 + 2*v11*v3 - 3*v11*v8*v9 + v11*v8 + 3*v11*v9 - 3*v11 + 7*v12*v13*v14*v15 - 3*v12*v13*v14 - 3*v12*v13*v15 + 8*v12*v13*

#### 5. Degree Reduction

In [48]:
def is_Number(x):
  try:
    a = float(x)
    return True
  except:
    return False

qubo_poly = ''
polys_list = str(poly).split(' ')
for i in range(0, len(polys_list),2):
  before = qubo_poly
  if polys_list[i].count('v') > 2:
    # 필요함
    if i != 0:
      if polys_list[i-1] == '-':
        # freedman method
        
        # 원본 식
        var_list.append(sym.Symbol(f'v{len(var_list)}'))
        tmp_poly_list = polys_list[i].split('*')
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*{var_list[-1]}*({polys_list[i].count("v")-1}'
          for j in range(1, len(tmp_poly_list)):
            qubo_poly += f'-{tmp_poly_list[j]}'
        else:
          qubo_poly += f'+{var_list[-1]}*({polys_list[i].count("v")-1}'
          for j in range(0, len(tmp_poly_list)):
            qubo_poly += f'-{tmp_poly_list[j]}'
        qubo_poly += ')'
      else:
        # Ishikawa method
        if polys_list[i].count('v') % 2 == 0:
          tmp_poly_list = polys_list[i].split('*')
          k = math.floor((polys_list[i].count('v') - 1) / 2)
          d = len(tmp_poly_list)
          varStart = len(var_list)
          for _ in range(k):
            var_list.append(sym.Symbol(f'v{len(var_list)}'))
          if(is_Number(tmp_poly_list[0])):
            qubo_poly += f'+{tmp_poly_list[0]}*(0'
            for n1 in range(1,d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d-1):
                qubo_poly += f'-2*{tmp_poly_list[1+n2]}*{var_list[varStart + n1]}'
            qubo_poly += ')'
          else:
            qubo_poly += f'+(0'
            for n1 in range(d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += ')'
        else:
          tmp_poly_list = polys_list[i].split('*')
          k = math.floor((polys_list[i].count('v') - 1) / 2)
          d = len(tmp_poly_list)
          varStart = len(var_list)
          for _ in range(k):
            var_list.append(sym.Symbol(f'v{len(var_list)}'))
          if(is_Number(tmp_poly_list[0])):
            qubo_poly += f'+{tmp_poly_list[0]}*(0'
            for n1 in range(1,d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d-1):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += f'+{var_list[varStart + k]}*(0'
            for n1 in range(d-1):
              qubo_poly += f'+{tmp_poly_list[n1]}'
            qubo_poly += f'-{d-1}+1))'
          else:
            qubo_poly += f'+(0'
            for n1 in range(d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += f'+{var_list[varStart + k]}*(0'
            for n1 in range(d):
              qubo_poly += f'+{tmp_poly_list[n1]}'
            qubo_poly += f'-{d}+1))'
    else:
      # Ishikawa method
      if polys_list[i].count('v') % 2 == 0:
        tmp_poly_list = polys_list[i].split('*')
        k = math.floor((polys_list[i].count('v') - 1) / 2)
        d = len(tmp_poly_list)
        varStart = len(var_list)
        for _ in range(k):
          var_list.append(sym.Symbol(f'v{len(var_list)}'))
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*(0'
          for n1 in range(1,d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d-1):
              qubo_poly += f'-2*{tmp_poly_list[1+n2]}*{var_list[varStart + n1]}'
          qubo_poly += ')'
        else:
          qubo_poly += f'+(0'
          for n1 in range(d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += ')'
      else:
        tmp_poly_list = polys_list[i].split('*')
        k = math.floor((polys_list[i].count('v') - 1) / 2)
        d = len(tmp_poly_list)
        varStart = len(var_list)
        for _ in range(k):
          var_list.append(sym.Symbol(f'v{len(var_list)}'))
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*(0'
          for n1 in range(1,d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d-1):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += f'+{var_list[varStart + k]}*(0'
          for n1 in range(d-1):
            qubo_poly += f'+{tmp_poly_list[n1]}'
          qubo_poly += f'-{d-1}+1))'
        else:
          qubo_poly += f'+(0'
          for n1 in range(d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += f'+{var_list[varStart + k]}*(0'
          for n1 in range(d):
            qubo_poly += f'+{tmp_poly_list[n1]}'
          qubo_poly += f'-{d}+1))'
  else:
    if i != 0:
      qubo_poly += polys_list[i-1]
    qubo_poly += polys_list[i]
  # print(f'{polys[i]} : {qubo_poly[len(before):]}')
qubo_poly = sym.sympify(qubo_poly)
# print(qubo_poly)
qubo_poly_extended = qubo_poly.expand()
qubo_poly = qubo_poly_extended.simplify()
print("변환한 식 변수의 수 : ", len(var_list))
# print(sym.solve(qubo_poly));
del qubo_poly_extended
print(qubo_poly)

변환한 식 변수의 수 :  96
36*v0*v1 - 14*v0*v16 - 3*v0*v17 - 3*v0*v18 - 16*v0*v19 + 10*v0*v2 - 4*v0*v20 - 4*v0*v21 - 16*v0*v22 - 4*v0*v23 - 4*v0*v24 - 16*v0*v25 - 2*v0*v26 - 2*v0*v27 - 3*v0*v28 - 4*v0*v29 + 8*v0*v3 - 4*v0*v30 - 2*v0*v31 + 12*v0*v4 + 10*v0*v5 + 12*v0*v6 + 10*v0*v7 + 10*v0*v8 + 8*v0*v9 - 3*v0 - 14*v1*v16 - 3*v1*v17 - 3*v1*v18 - 16*v1*v19 + 8*v1*v2 - 4*v1*v20 - 4*v1*v21 - 16*v1*v22 - 4*v1*v23 - 4*v1*v24 - 16*v1*v25 - 2*v1*v26 - 2*v1*v27 + 10*v1*v3 - 3*v1*v32 - 4*v1*v33 - 4*v1*v34 - 2*v1*v35 + 10*v1*v4 + 12*v1*v5 + 10*v1*v6 + 12*v1*v7 + 8*v1*v8 + 10*v1*v9 - 3*v1 + 36*v10*v11 + 12*v10*v12 + 10*v10*v13 + 12*v10*v14 + 10*v10*v15 + 10*v10*v2 + 8*v10*v3 - 16*v10*v36 - 4*v10*v37 - 4*v10*v38 - 16*v10*v39 - 4*v10*v40 - 4*v10*v41 - 16*v10*v42 - 2*v10*v43 - 2*v10*v44 - 14*v10*v45 - 3*v10*v46 - 3*v10*v47 - 4*v10*v48 - 4*v10*v49 - 2*v10*v50 - 3*v10*v51 + 10*v10*v8 + 8*v10*v9 - 3*v10 + 10*v11*v12 + 12*v11*v13 + 10*v11*v14 + 12*v11*v15 + 8*v11*v2 + 10*v11*v3 - 16*v11*v36 - 4*v11*v37 - 4*v11*v38 

#### 6. dimod 모듈의 적합한 형태로 변화 및 데이터 저장

In [49]:
terms = {}

for i in poly.args:
  new_poly = str(i).split('*')
  if(new_poly[0].startswith('-v')):
    new_poly.insert(0, '-1')
    new_poly[1] = new_poly[1][1:]
  elif(new_poly[0].startswith('v')):
    new_poly.insert(0, '1')
  if(len(new_poly)==1):
    pass
  elif(len(new_poly)==2):
    terms[('v' + str(new_poly[1][1:]), 'v' + str(new_poly[1][1:]))] = float(new_poly[0])
  elif(len(new_poly)==3):
    terms[('v' + str(new_poly[1][1:]), 'v' + str(new_poly[2][1:]))] = float(new_poly[0])

with open("data.pickle","wb") as fw:
    pickle.dump(terms, fw)